In [1]:
import pandas as pd
import time
import urllib.request
import xml.etree.ElementTree as ET
import jellyfish #mit
import re
import numpy as np
import rapidfuzz #mit
import xmltodict

First read client data from excel

In [2]:
program_start = time.time()
start = time.time()
path = 'C:/Users/Aleksi/Downloads/example_names.xlsx'
#Read data in from excel file
client_data = pd.read_excel(path)
#divide into first and last name for further calculations
client_data['name'] = client_data['name'].str.upper()
client_data['name_parts'] = client_data['name'].str.split(' |-')
print(client_data)
end = time.time()
elapsed_time = end-start
print('elapsed time in reading client data:',elapsed_time)

       row_id                    name                    name_parts
0           1   MERJA-MAARIT PERHONEN     [MERJA, MAARIT, PERHONEN]
1           2           HEIKO PISTOOL              [HEIKO, PISTOOL]
2           3    JAANA-MAIJA UUSPELTO      [JAANA, MAIJA, UUSPELTO]
3           4       NATNICHA MITJONEN          [NATNICHA, MITJONEN]
4           5           KATHRIN FLYKT              [KATHRIN, FLYKT]
...       ...                     ...                           ...
50016   50017        ALEXINA SARENIUS           [ALEXINA, SARENIUS]
50017   50018        ABDISAMED NAZARI           [ABDISAMED, NAZARI]
50018   50019            SONER JUUDIN               [SONER, JUUDIN]
50019   50020          PIETRO JUURELA             [PIETRO, JUURELA]
50020   50021  ABD AL-KHALIQ AL-HUTHI  [ABD, AL, KHALIQ, AL, HUTHI]

[50021 rows x 3 columns]
elapsed time in reading client data: 3.356520891189575


Read in the xlm

In [3]:
start = time.time()
xml_url = "https://scsanctions.un.org/resources/xml/en/consolidated.xml"
response = urllib.request.urlopen(xml_url)
tree = ET.parse(response)

df_sanctioned_individuals = pd.DataFrame()
#iterate through all individuals in sanctionlist
for individual in tree.findall(".//INDIVIDUAL"):
    features = {}
    #save all the data
    for node in individual.iter():
        features[node.tag] = node.text
    #only saves the last alias without this piece of code
    alias_names = []
    for alias in individual.findall('.//INDIVIDUAL_ALIAS'):
        if alias.find('QUALITY').text == 'Good':
            alias_names.append(alias.find('ALIAS_NAME').text)
    features['ALIAS_NAME'] = ', '.join(alias_names)
    
    df_features = pd.DataFrame(features, index=['0'])
    df_sanctioned_individuals = df_sanctioned_individuals.append(df_features, ignore_index=True)
df_sanctioned_individuals['Full Name'] = df_sanctioned_individuals[['FIRST_NAME', 'SECOND_NAME', 'THIRD_NAME', 'FOURTH_NAME']].fillna('').apply(lambda x: ' '.join(x), axis=1)
#this removes the spaces from the end and begginnig
df_sanctioned_individuals['Full Name'] = df_sanctioned_individuals['Full Name'].str.strip()
#create csv file of the dataframe
df_sanctioned_individuals.to_csv('UN_sanction_data.csv', index=False)

print(df_sanctioned_individuals[:5])

end = time.time()
elapsed_time = end - start
print('elapsed time in reading xlm:', elapsed_time)

C:\Users\Aleksi\Anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


       ALIAS_NAME CITY CITY_OF_BIRTH CITY_OF_ISSUE  \
0                  NaN           NaN           NaN   
1   Jang Chang Ha  NaN           NaN           NaN   
2  Jo Chun Ryong   NaN           NaN           NaN   
3                  NaN           NaN           NaN   
4   Cho Yong Chol  NaN           NaN           NaN   

                                           COMMENTS1              COUNTRY  \
0  Ri Won Ho is a DPRK Ministry of State Security...                 None   
1                                               None                 None   
2                                               None                 None   
3  Senior member of Islamic State in Iraq and the...  Trinidad and Tobago   
4  Jo Yong Chol is a DPRK Ministry of State Secur...                 None   

  COUNTRY_OF_BIRTH COUNTRY_OF_ISSUE   DATAID        DATE  ... TO_YEAR  \
0              NaN              NaN  6908555  1964-07-17  ...     NaN   
1              NaN              NaN  6908570  1964-01-10  ...     

In [4]:
start = time.time()
#matches tokens first then compares the nearest tokens with indel algortihm (levenstein but change cost 2(delete + insert))
#running with cdist makes it 3 times faster with set of 50 000
distances_names = rapidfuzz.process.cdist(client_data['name'], df_sanctioned_individuals['Full Name'], scorer = rapidfuzz.fuzz.token_set_ratio, workers =-1, score_hint=75)
distances_aliases = rapidfuzz.process.cdist(client_data['name'], df_sanctioned_individuals['ALIAS_NAME'], scorer = rapidfuzz.fuzz.token_set_ratio, workers =-1)
#save results into dataframe
distances_names_df = pd.DataFrame(distances_names, index=client_data['name'], columns=df_sanctioned_individuals['Full Name'])
distances_aliases_df = pd.DataFrame(distances_aliases, index=client_data['name'], columns=df_sanctioned_individuals['ALIAS_NAME'])
#filter with score over 80
name_scores_over_80 = distances_names_df[distances_names_df > 80].stack()
alias_scores_over_80 = distances_aliases_df[distances_aliases_df > 80].stack()

print("We only display first 10 hits to save space\n", name_scores_over_80[:10])
print("We only display first 10 hits to save space\n", alias_scores_over_80[:10])

end = time.time()
elapsed_time = end - start
print('elapsed time in token set algorithm:', elapsed_time)


We only display first 10 hits to save space
 name                 Full Name                      
BI SIDI SOULEMAN     BI SIDI  SOULEMAN                  100.000000
MOHAMMAD SAND        MOHAMMAD ZAHID                      81.481483
KIM MUN COL          KIM MUN CHOL                        95.652176
MARIAL CHANUONG      MARIAL CHANUONG YOL MANGOK         100.000000
GHITA AHMED MOHAMED  MOHAMED BEN AHMED MAHRI             81.250000
SIN AL-AZZAWI        HIKMAT MIZBAN IBRAHIM AL-AZZAWI     81.818184
YAHYA HAQQANI        YAHYA HAQQANI                      100.000000
MOHAMMAD SAHARI      MOHAMMAD AHMADI                     86.666664
                     MOHAMMAD ZAHID                      82.758621
KADRIYE MOHAMED ALI  ALI MOHAMED RAGE                    81.481483
dtype: float32
We only display first 10 hits to save space
 Series([], dtype: float32)
elapsed time in token set algorithm: 105.08374691009521


Calculate soundex for all the sanctioned individuals

In [5]:
start = end = time.time()
# sanctioned individuals
df_sanctioned_individuals['name_parts'] = df_sanctioned_individuals['Full Name'].apply(lambda x: re.split(' |-', x))
df_sanctioned_individuals['name_parts_soundex'] = df_sanctioned_individuals['name_parts'].apply(lambda x: ' '.join([jellyfish.soundex(name) for name in x]) if len(x) > 1 else '')

print(df_sanctioned_individuals[['name_parts', 'name_parts_soundex']].iloc[0:3])

# users
client_data['name_parts'] = client_data['name'].apply(lambda x: re.split(' |-', x))    
client_data['name_parts_soundex'] = client_data['name_parts'].apply(lambda x: ' '.join(jellyfish.soundex(name) for name in x))
print(client_data[['name_parts', 'name_parts_soundex']].iloc[0:3])

end = time.time()
elapsed_time = end - start
print('elapsed time in transforming names into soundex:', elapsed_time)

               name_parts name_parts_soundex
0         [RI, , WON, HO]    R000  W500 H000
1      [CHANG, CHANG, HA]     C520 C520 H000
2  [CHO, , , CHUN, RYONG]   C000   C500 R520
                  name_parts name_parts_soundex
0  [MERJA, MAARIT, PERHONEN]     M620 M630 P655
1           [HEIKO, PISTOOL]          H200 P234
2   [JAANA, MAIJA, UUSPELTO]     J500 M200 U214
elapsed time in transforming names into soundex: 0.4252147674560547


Soundex compare UN list with our list

In [6]:
start = end = time.time()
#matches tokens first then compares the nearest tokens with indel algortihm (levenstein but change cost 2(delete + insert))
distances = rapidfuzz.process.cdist(client_data['name_parts_soundex'], df_sanctioned_individuals['name_parts_soundex'], scorer = rapidfuzz.fuzz.token_set_ratio, workers =-1, score_hint=100)

scores_100_soundex = pd.DataFrame(distances, index=client_data['name'], columns=df_sanctioned_individuals['Full Name'])
scores_100_soundex = scores_100_soundex[scores_100_soundex == 100].stack()

print(scores_100_soundex)
end = time.time()
elapsed_time = end - start
print('elapsed time in token set algorithm for soundex:', elapsed_time)

name                    Full Name                               
BI SIDI SOULEMAN        BI SIDI  SOULEMAN                           100.0
ANDRUS ANTEROINEN       ANDERS CAMEROON OSTENSVIG DALE              100.0
MACO ALA-OLLA           ALI MAYCHOU                                 100.0
BAYAN SUITTIO           SA'D BIN SA'D MUHAMMAD SHARIYAN AL-KA'BI    100.0
                        MAHRI SIDI AMAR BEN DAHA                    100.0
                                                                    ...  
MEHMET TAPIO            ALLAH DAD TAYEB WALI MUHAMMAD               100.0
ROSALIA REKOLA          MOHAMMAD RASUL AYYUB                        100.0
HAMED KURRU             HAMADA OULD MOHAMED EL KHAIRY               100.0
TEIJU TOHKA             QARI SAIFULLAH TOKHI                        100.0
ABD AL-KHALIQ AL-HUTHI  ABD AL-KHALIQ  AL-HOUTHI                    100.0
Length: 771, dtype: float32
elapsed time in token set algorithm for soundex: 47.352813482284546


In [7]:
writer = pd.ExcelWriter('SanctionScreenResults.xlsx', engine='xlsxwriter')

# write each dataframe to a separate worksheet
name_scores_over_80.to_excel(writer, sheet_name='Name hits token set ratio')
#alias_scores_over_80.to_excel(writer, sheet_name='Alias hits token set ratio')
scores_100_soundex.to_excel(writer, sheet_name='Soundex Results')
# save the writer
writer.save()

program_end = time.time()
elapsed_program_time = program_end-program_start
print('elapsed time for whole progam:',elapsed_program_time)

elapsed time for whole progam: 171.19136667251587


This is a function for reading a single name in 

In [10]:
def calculate_single(name):
    start = time.time()
    #matches tokens first then compares the nearest tokens with indel algortihm (levenstein but change cost 2(delete + insert))
    #running with cdist makes it 3 times faster with set of 50 000
    distances_names = rapidfuzz.process.cdist([name], df_sanctioned_individuals['Full Name'], scorer = rapidfuzz.fuzz.token_set_ratio, workers =-1, score_hint=75)
    distances_aliases = rapidfuzz.process.cdist([name], df_sanctioned_individuals['ALIAS_NAME'], scorer = rapidfuzz.fuzz.token_set_ratio, workers =-1)
    #save results into dataframe
    distances_names_df = pd.DataFrame(distances_names, index=[name], columns=df_sanctioned_individuals['Full Name'])
    distances_aliases_df = pd.DataFrame(distances_aliases, index=[name], columns=df_sanctioned_individuals['ALIAS_NAME'])
    #filter with score over 80
    name_scores_over_80 = distances_names_df[distances_names_df > 80].stack()
    alias_scores_over_80 = distances_aliases_df[distances_aliases_df > 80].stack()

    print("We only display first 10 hits to save space\n", name_scores_over_80[:10])
    print("We only display first 10 hits to save space\n", alias_scores_over_80[:10])

    end = time.time()
    elapsed_time = end - start
    print('elapsed time in token set algorithm:', elapsed_time)
    return (name_scores_over_80,alias_scores_over_80)


df_namescore = pd.DataFrame()
df_alias_score = pd.DataFrame()
df_namescore,df_alias_score=calculate_single("BI SIDI SOULEMAN")


We only display first 10 hits to save space
                   Full Name        
BI SIDI SOULEMAN  BI SIDI  SOULEMAN    100.0
dtype: float32
We only display first 10 hits to save space
 Series([], dtype: float32)
elapsed time in token set algorithm: 0.6109671592712402


A slower version but can be used to double check results

In [ ]:


start = end = time.time()
# Define a function to calculate the distance between a client and a sanction list suspect
def calculate_distance(client, suspect_list):
    #distances = np.vectorize(rapidfuzz.fuzz.token_sort_ratio)(client, suspect_list)
    distances = np.vectorize(rapidfuzz.fuzz.token_set_ratio)(client, suspect_list)
    max_distance = distances.max()
    max_suspect = suspect_list[distances.argmax()]
    return max_distance, max_suspect
# Apply the function to each row of the client_data DataFrame
client_data['max_distance_token_set'] = 0
client_data['max_suspect_token_set'] = ''
client_data['max_distance_token_set'], client_data['max_suspect_token_set'] = zip(*client_data['name'].apply(lambda x: calculate_distance(x, df_sanctioned_individuals['Full Name'])))

filtered_data_token_set = client_data[client_data['max_distance_token_set']>80]
print(filtered_data_token_set[['name','max_suspect_token_set', 'max_distance_token_set']])

end = time.time()
elapsed_time = end - start
print('elapsed time in token set algorithm:', elapsed_time)